In [13]:
import tensorflow as tf
import tensorflow_datasets as tfds

tf.__version__

'2.19.0'

In [12]:
# load the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

len(x_train) , len(x_test)

(60000, 10000)

In [73]:
datasets, info  = tfds.load(name="mnist", split=["train","test"] , with_info=True, as_supervised=True)


In [77]:
datasets

[<_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>]

In [78]:
mninst_train, mnist_test = datasets[0],  datasets[1]

In [79]:
mninst_train

<_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [84]:
# Data scaling
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image = image/ 255.0
    return image, label

In [91]:
strategy = tf.distribute.MirroredStrategy()

strategy.num_replicas_in_sync

BATCH_SIZE = 64 * strategy.num_replicas_in_sync

train_dataset = mninst_train.map(scale).cache().shuffle(10000).batch(BATCH_SIZE)
test_dataset = mnist_test.map(scale).batch(BATCH_SIZE)





INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [92]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(28,28,1)),
        tf.keras.layers.Conv2D(32, 3, activation="relu"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(10)
    ])
    
    